In [12]:
import sqlite3
from autogc_validation.database.schema.create_table import create_table
from autogc_validation.database.schema.schemas import SCHEMAS
from autogc_validation.database.connection.manager import get_connection
import pandas as pd
import logging
import tempfile
import os


In [13]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [14]:
with tempfile.NamedTemporaryFile(suffix=".db", delete=False) as tmpfile:
    db_path = tmpfile.name

logger.info(f"Temporary database path: {db_path}")

def show_tables(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [row[0] for row in cursor.fetchall()]

# -----------------------------
# Create tables and inspect
# -----------------------------
with get_connection(db_path) as conn:
    # Create tables from SCHEMAS
    for table_name, sql in SCHEMAS.items():
        logger.info(f"Creating table: {table_name}")
        conn.execute(sql.sql)

    tables = show_tables(conn)
    print("Tables in DB:", tables)

# -----------------------------
# Optional: Clean up
# -----------------------------
os.remove(db_path)
logger.info("Temporary database deleted.")

INFO:__main__:Temporary database path: C:\Users\AENGST~1\AppData\Local\Temp\tmpn8snfdq4.db
INFO:__main__:Creating table: schema_version
INFO:__main__:Creating table: voc_info
INFO:__main__:Creating table: sites
INFO:__main__:Creating table: canister_types
INFO:__main__:Creating table: primary_canisters
INFO:__main__:Creating table: primary_canister_concentration
INFO:__main__:Creating table: site_canisters
INFO:__main__:Creating table: mdls
INFO:__main__:Creating table: site_canister_concentration_view
INFO:__main__:Temporary database deleted.


Tables in DB: ['SchemaVersion', 'voc_info', 'sites', 'canister_types', 'primary_canisters', 'primary_canister_concentration', 'site_canisters', 'mdls']


# Test table creation

In [1]:
import sqlite3
import pandas as pd
import logging
import tempfile
import os
from autogc_validation.database.management.init_db import initialize_database
from autogc_validation.database.operations.voc_info import get_all_voc_data
from autogc_validation.database.operations.insert import insert
from autogc_validation.database.operations.get_table import get_table
import autogc_validation.database.models as models


In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
with tempfile.NamedTemporaryFile(suffix=".db", delete=False) as tmpfile:
    db_path = tmpfile.name

logger.info(f"Temporary database path: {db_path}")
initialize_database(db_path, force = True)
voc_df = get_table(db_path, "voc_info", order_by = ["column", "elution_order"])
compound_to_code = dict(zip(voc_df["compound"], voc_df["aqs_code"]))
compound_to_code



INFO:__main__:Temporary database path: C:\Users\AENGST~1\AppData\Local\Temp\tmp4t7qv4jm.db
INFO:autogc_validation.database.management.init_db:Deleted existing database
INFO:autogc_validation.database.management.init_db:Initializing database at C:\Users\AENGST~1\AppData\Local\Temp\tmp4t7qv4jm.db
INFO:autogc_validation.database.management.init_db:Creating tables...
INFO:autogc_validation.database.operations.create_table:Created table sites
INFO:autogc_validation.database.operations.create_table:Created table voc_info
INFO:autogc_validation.database.operations.create_table:Created table canister_types
INFO:autogc_validation.database.operations.create_table:Created table primary_canisters
INFO:autogc_validation.database.operations.create_table:Created table primary_canister_concentration
INFO:autogc_validation.database.operations.create_table:Created table site_canisters
INFO:autogc_validation.database.operations.create_table:Created table mdls
INFO:autogc_validation.database.operations.cr

{'N-hexane': 43231,
 'Methylcyclopentane': 43262,
 '2,4-dimethylpentane': 43247,
 'Benzene': 45201,
 'Cyclohexane': 43248,
 '2-methylhexane': 43263,
 '2,3-dimethylpentane': 43291,
 '3-methylhexane': 43249,
 '2,2,4-trimethylpentane': 43250,
 'N-heptane': 43232,
 'Methylcyclohexane': 43261,
 '2,3,4-trimethylpentane': 43252,
 'Toluene': 45202,
 '2-methylheptane': 43960,
 '3-methylheptane': 43253,
 'N-octane': 43233,
 'Ethylbenzene': 45203,
 'M&p-xylene': 45109,
 'Styrene': 45220,
 'O-xylene': 45204,
 'N-nonane': 43235,
 'Iso-propylbenzene': 45210,
 'Alpha-pinene': 43256,
 'N-propylbenzene': 45209,
 'M-ethyltoluene': 45212,
 'P-ethyltoluene': 45213,
 '1,3,5-tri-m-benzene': 45207,
 'O-ethyltoluene': 45211,
 'Beta-pinene': 43257,
 '1,2,4-tri-m-benzene': 45208,
 'N-decane': 43238,
 '1,2,3-tri-m-benzene': 45225,
 'M-diethylbenzene': 45218,
 'P-diethylbenzene': 45219,
 'N-undecane': 43954,
 'N-dodecane': 43141,
 'Ethane': 43202,
 'Ethylene': 43203,
 'Propane': 43204,
 'Propylene': 43205,
 'Iso-

## Test Insertion

In [5]:

s1 = models.Site(490353014, 'LP', 'Lake Park', 40.709905, -112.008684, '2025-02-06 10:00:00')  
insert(db_path, s1)
site = get_table(db_path, s1.__tablename__)

cvs = models.CanisterTypes("CVS")
insert(db_path, cvs)
cvs_sql = get_table(db_path, cvs.__tablename__)
pcan = models.PrimaryCanister("CC524930-0626", "CVS", '2026-06-01 00:00:00')
insert(db_path, pcan)
can = get_table(db_path, pcan.__tablename__)
rb_cvs = {'Ethane': .525*2*1000,
                                     'Propane': 0.34*3*1000,
                                     'N-butane': 0.253*4*1000,
                                     'Acetylene': 0.525*2*1000,
                                     'N-pentane': 0.204*5*1000,
                                     '1,3-butadiene': 0.263*4*1000,
                                     '2-methylpentane': 0.17*6*1000,
                                     '1-hexene': 0.17*6*1000,
                                     'N-hexane': 0.167*6*1000,
                                     'Benzene': 0.175*6*1000,
                                     'Toluene': 0.146*7*1000,
                                     'M&p-xylene': 0.131*8*1000,
                                     'N-propylbenzene': 0.116*9*1000,
                                     '1,2,4-tri-m-benzene': 0.113*9*1000,
                                     'P-diethylbenzene': 0.102*10*1000}
cvs_conc = {compound_to_code.get(key): value for key, value in rb_cvs.items()}
can_name = "CC524930-0626"
can_type = "CVS"
for compound, conc in cvs_conc.items():
    c = models.CanisterConcentration(can_name, int(compound), conc, "ppbc", can_type)
    insert(db_path, c)
conc_table = get_table(db_path, c.__tablename__)
conc_table
lp_mdls = {
    "Ethane": 0.0926,
    "Ethylene": 0.0602,
    "Propane": 0.0932,
    "Propylene": 0.117,
    "Iso-butane": 0.0677,
    "N-butane": 0.0965,
    "Acetylene": 0.0586,
    "Trans-2-butene": 0.0498,
    "1-butene": 0.0389,
    "Cis-2-butene": 0.0476,
    "Cyclopentane": 0.0616,
    "Iso-pentane": 0.0646,
    "N-pentane": 0.0473,
    "1,3-butadiene": 0.0511,
    "Trans-2-pentene": 0.0517,
    "1-pentene": 0.0398,
    "Cis-2-pentene": 0.0404,
    "2,2-dimethylbutane": 0.06,
    "2,3-dimethylbutane": 0.0547,
    "2-methylpentane": 0.0399,
    "3-methylpentane": 0.0442,
    "Isoprene": 0.032,
    "2-methyl-1-pentene": 0.0586,
    "1-hexene": 0.0393,
    "N-hexane": 0.0844,
    "Methylcyclopentane": 0.0729,
    "2,4-dimethylpentane": 0.09,
    "Benzene": 0.2199,
    "Cyclohexane": 0.1409,
    "2-methylhexane": 0.095,
    "2,3-dimethylpentane": 0.1452,
    "3-methylhexane": 0.1393,
    "2,2,4-trimethylpentane": 0.0855,
    "N-heptane": 0.0869,
    "Methylcyclohexane": 0.0629,
    "2,3,4-trimethylpentane": 0.0711,
    "Toluene": 0.0952,
    "2-methylheptane": 0.0665,
    "3-methylheptane": 0.0797,
    "N-octane": 0.1424,
    "Ethylbenzene": 0.0837,
    "M&p-xylene": 0.0707,
    "Styrene": 0.1378,
    "O-xylene": 0.0766,
    "N-nonane": 0.0775,
    "Iso-propylbenzene": 0.0735,
    "Alpha-pinene": 0.2379,
    "N-propylbenzene": 0.083,
    "M-ethyltoluene": 0.1292,
    "P-ethyltoluene": 0.1249,
    "1,3,5-tri-m-benzene": 0.1323,
    "O-ethyltoluene": 0.1433,
    "Beta-pinene": 0.1138,
    "1,2,4-tri-m-benzene": 0.1162,
    "N-decane": 0.1104,
    "1,2,3-tri-m-benzene": 0.1494,
    "M-diethylbenzene": 0.1365,
    "P-diethylbenzene": 0.1762,
    "N-undecane": 0.1428,
    "N-dodecane": 0.2269
}
lp_mdls = {compound_to_code.get(key): value for key, value in lp_mdls.items()}
print(lp_mdls)
date_on = "2025-10-01 00:00:00"
date_off = "2025-09-01 00:00:00"
for compound, mdl in lp_mdls.items():
    print(compound)
    a = models.MDL(s1.site_id, compound, mdl, date_on, date_off)
    insert(db_path, a)
mdl_table = get_table(db_path, a.__tablename__)
mdl_table
#cvs_conc = CanisterConcentration("CC524930-0626",

{43202: 0.0926, 43203: 0.0602, 43204: 0.0932, 43205: 0.117, 43214: 0.0677, 43212: 0.0965, 43206: 0.0586, 43216: 0.0498, 43280: 0.0389, 43217: 0.0476, 43242: 0.0616, 43221: 0.0646, 43220: 0.0473, 43218: 0.0511, 43226: 0.0517, 43224: 0.0398, 43227: 0.0404, 43244: 0.06, 43284: 0.0547, 43285: 0.0399, 43230: 0.0442, 43243: 0.032, 43246: 0.0586, 43245: 0.0393, 43231: 0.0844, 43262: 0.0729, 43247: 0.09, 45201: 0.2199, 43248: 0.1409, 43263: 0.095, 43291: 0.1452, 43249: 0.1393, 43250: 0.0855, 43232: 0.0869, 43261: 0.0629, 43252: 0.0711, 45202: 0.0952, 43960: 0.0665, 43253: 0.0797, 43233: 0.1424, 45203: 0.0837, 45109: 0.0707, 45220: 0.1378, 45204: 0.0766, 43235: 0.0775, 45210: 0.0735, 43256: 0.2379, 45209: 0.083, 45212: 0.1292, 45213: 0.1249, 45207: 0.1323, 45211: 0.1433, 43257: 0.1138, 45208: 0.1162, 43238: 0.1104, 45225: 0.1494, 45218: 0.1365, 45219: 0.1762, 43954: 0.1428, 43141: 0.2269}
43202


ValidationError: 1 validation error for MDL
  Value error, date_on must be before date_off [type=value_error, input_value=ArgsKwargs((490353014, 43... '2025-09-01 00:00:00')), input_type=ArgsKwargs]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [5]:
os.remove(db_path)
logger.info("Temporary database deleted.")

INFO:__main__:Temporary database deleted.


In [1]:
from autogc_validation.database.models.registry import MODEL_REGISTRY
MODEL_REGISTRY

{'sites': <class 'autogc_validation.database.models.site.Site'>, 'voc_info': <class 'autogc_validation.database.models.voc.VOCInfo'>, 'primary_canisters': <class 'autogc_validation.database.models.canister.PrimaryCanister'>, 'primary_canister_concentration': <class 'autogc_validation.database.models.canister.CanisterConcentration'>, 'site_canisters': <class 'autogc_validation.database.models.canister.SiteCanister'>, 'mdls': <class 'autogc_validation.database.models.mdl.MDL'>, 'SchemaVersion': <class 'autogc_validation.database.models.version.Version'>}


{'sites': autogc_validation.database.models.site.Site,
 'voc_info': autogc_validation.database.models.voc.VOCInfo,
 'primary_canisters': autogc_validation.database.models.canister.PrimaryCanister,
 'primary_canister_concentration': autogc_validation.database.models.canister.CanisterConcentration,
 'site_canisters': autogc_validation.database.models.canister.SiteCanister,
 'mdls': autogc_validation.database.models.mdl.MDL,
 'SchemaVersion': autogc_validation.database.models.version.Version}